# Data Ingestion

This notebook reads and ingest data from Olist (brazilian marketplace), for data analysis. 

For more information about the data source: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce/data

Import libraries

In [1]:
import kagglehub
from pyspark.sql import SparkSession 
from pyspark.sql import functions as F
from pyspark.sql import Window
import pandas as pd

c:\GitHubJFF\olist_data\ambiente_teste\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dowload raw data

In [2]:
# Download latest version
# path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")
# print("Path to dataset files:", path)

Initialize a Spark Session

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("OlistData") \
    .getOrCreate()

Read raw data

In [4]:
customers = spark.read.csv("data/raw_data/olist_customers_dataset.csv", header=True, inferSchema=True)

geolocation = spark.read.csv("data/raw_data/olist_geolocation_dataset.csv", header=True, inferSchema=True)

orders = spark.read.csv("data/raw_data/olist_orders_dataset.csv", header=True, inferSchema=True)

order_items = spark.read.csv("data/raw_data/olist_order_items_dataset.csv", header=True, inferSchema=True)

products = spark.read.csv("data/raw_data/olist_products_dataset.csv", header=True, inferSchema=True)

order_payments = spark.read.csv("data/raw_data/olist_order_payments_dataset.csv", header=True, inferSchema=True)

order_reviews = spark.read \
    .option("header", True) \
    .option("sep", ",") \
    .option("quote", '"') \
    .option("escape", '"') \
    .option("multiLine", True) \
    .option("inferSchema", True) \
    .csv("data/raw_data/olist_order_reviews_dataset.csv")

sellers = spark.read.csv("data/raw_data/olist_sellers_dataset.csv", header=True, inferSchema=True)

product_category_name = spark.read.csv("data/raw_data/product_category_name_translation.csv", header=True, inferSchema=True)

## Validations

In [5]:
# Function to verify unique keys
def verify_unique_key(df, key_column):
    total_rows = df.count()
    unique_keys = df.select(key_column).distinct().count()
    
    if total_rows == unique_keys:
        print(f"Column '{key_column}' is a unique key.")
        result = True
    else:
        print(f"Column '{key_column}' is NOT a unique key. Total rows: {total_rows}, Unique keys: {unique_keys}")
        result = False
        
    return result

In [6]:
# Function to show null values in all columns
def show_null_values(df):
    null_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns])
    null_counts.show()

### Customers

In [7]:
customers.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [8]:
customers.show(5)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
+--------------------+--------------------+------------------------+--------------------+--------------+
only showing top 5 rows



In [9]:
verify_unique_key(customers, "customer_id")

Column 'customer_id' is a unique key.


True

In [10]:
show_null_values(customers)

+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+



### Geolocation

In [11]:
geolocation.printSchema()

root
 |-- geolocation_zip_code_prefix: integer (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)



In [12]:
geolocation.show(5)

+---------------------------+-------------------+------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|   geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+------------------+----------------+-----------------+
|                       1037| -23.54562128115268|-46.63929204800168|       sao paulo|               SP|
|                       1046|-23.546081127035535|-46.64482029837157|       sao paulo|               SP|
|                       1046| -23.54612896641469|-46.64295148361138|       sao paulo|               SP|
|                       1041|  -23.5443921648681|-46.63949930627844|       sao paulo|               SP|
|                       1035|-23.541577961711493|-46.64160722329613|       sao paulo|               SP|
+---------------------------+-------------------+------------------+----------------+-----------------+
only showing top 5 rows



In [13]:
verify_unique_key(geolocation, "geolocation_zip_code_prefix")

Column 'geolocation_zip_code_prefix' is NOT a unique key. Total rows: 1000163, Unique keys: 19015


False

In [14]:
geolocation.orderBy("geolocation_zip_code_prefix").show(5)

+---------------------------+-------------------+-------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|    geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+-------------------+----------------+-----------------+
|                       1001| -23.55064182209015|-46.634409790322515|       sao paulo|               SP|
|                       1001| -23.54929199999999|-46.633559478233785|       sao paulo|               SP|
|                       1001|-23.550497706907514| -46.63433817805407|       sao paulo|               SP|
|                       1001| -23.54969829946912| -46.63390859285005|       sao paulo|               SP|
|                       1001|-23.551426655288804| -46.63407394670785|       sao paulo|               SP|
+---------------------------+-------------------+-------------------+----------------+-----------------+
only showing top 5 rows



In [15]:
geolocation = geolocation \
    .groupBy("geolocation_zip_code_prefix")\
    .agg(
        F.avg("geolocation_lat").alias("geolocation_lat"),
        F.avg("geolocation_lng").alias("geolocation_lng"),
        F.max("geolocation_city").alias("geolocation_city"),
        F.max("geolocation_state").alias("geolocation_state")
    )


In [16]:
verify_unique_key(geolocation, "geolocation_zip_code_prefix")

Column 'geolocation_zip_code_prefix' is a unique key.


True

In [17]:
show_null_values(geolocation)

+---------------------------+---------------+---------------+----------------+-----------------+
|geolocation_zip_code_prefix|geolocation_lat|geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+---------------+---------------+----------------+-----------------+
|                          0|              0|              0|               0|                0|
+---------------------------+---------------+---------------+----------------+-----------------+



### Orders

In [18]:
orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [19]:
orders.show(5)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [20]:
verify_unique_key(orders, "order_id")

Column 'order_id' is a unique key.


True

In [21]:
show_null_values(orders)

+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|order_id|customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|       0|          0|           0|                       0|              160|                        1783|                         2965|                            0|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+



In [22]:
count_nulls_status = orders\
    .groupBy("order_status")\
    .agg(
        F.count(F.when(F.col("order_approved_at").isNull(), "order_approved_null")).alias("order_approved_null"),
        F.count(F.when(F.col("order_delivered_carrier_date").isNull(), "order_delivered_carrier_null")).alias("order_delivered_carrier_null"),
        F.count(F.when(F.col("order_delivered_customer_date").isNull(), "order_delivered_customer_null")).alias("order_delivered_customer_null"),
    )
count_nulls_status.show()

+------------+-------------------+----------------------------+-----------------------------+
|order_status|order_approved_null|order_delivered_carrier_null|order_delivered_customer_null|
+------------+-------------------+----------------------------+-----------------------------+
|     shipped|                  0|                           0|                         1107|
|    canceled|                141|                         550|                          619|
|    invoiced|                  0|                         314|                          314|
|     created|                  5|                           5|                            5|
|   delivered|                 14|                           2|                            8|
| unavailable|                  0|                         609|                          609|
|  processing|                  0|                         301|                          301|
|    approved|                  0|                          

In [23]:
total_orders_status = orders.groupBy("order_status").count()
total_orders_status.show()

+------------+-----+
|order_status|count|
+------------+-----+
|     shipped| 1107|
|    canceled|  625|
|    invoiced|  314|
|     created|    5|
|   delivered|96478|
| unavailable|  609|
|  processing|  301|
|    approved|    2|
+------------+-----+



In [24]:
# Unifying total with null count to understand inconsistencies in the data
orders_nulls_analysis = total_orders_status.join(count_nulls_status, on="order_status", how="left")
orders_nulls_analysis.show()

+------------+-----+-------------------+----------------------------+-----------------------------+
|order_status|count|order_approved_null|order_delivered_carrier_null|order_delivered_customer_null|
+------------+-----+-------------------+----------------------------+-----------------------------+
|     shipped| 1107|                  0|                           0|                         1107|
|    canceled|  625|                141|                         550|                          619|
|    invoiced|  314|                  0|                         314|                          314|
|     created|    5|                  5|                           5|                            5|
|   delivered|96478|                 14|                           2|                            8|
| unavailable|  609|                  0|                         609|                          609|
|  processing|  301|                  0|                         301|                          301|


In [25]:
# Mapping registries that has inconsistencies in the data

# Registries that has been delivered but has no approving, carrier delivering, or customer delivering date
orders = orders \
    .withColumn(
        "inconsistency_delivered_null",
        F.when(
            (F.col("order_status") == "delivered") &
            (
                F.col("order_approved_at").isNull() |
                F.col("order_delivered_carrier_date").isNull() |
                F.col("order_delivered_customer_date").isNull()
            ),
            1
        ).otherwise(0)
    )
    
orders.filter(F.col("inconsistency_delivered_null") == 1).show(30)


+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|inconsistency_delivered_null|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------------------+
|2d1e2d5bf4dc7227b...|ec05a6d8558c6455f...|   delivered|     2017-11-28 17:44:07|2017-11-28 17:56:40|         2017-11-30 18:12:23|                         NULL|          2017-12-18 00:00:00|                           1|
|e04abd8149ef81b95...|2127dc6603ac33544...|   delivered|     2017-02-18 14:40:00|               NULL|         2017-02-23

In [26]:
inconsistency_percentage = orders.filter(F.col("inconsistency_delivered_null") == 1).count() / orders.count() * 100
print(f"Inconsistency percentage for delivered orders with null dates: {inconsistency_percentage:.2f}%")

Inconsistency percentage for delivered orders with null dates: 0.02%


In [27]:
# Dropping the inconsistency column as it is not relevant for the analysis and has a low percentage of affected records
orders = orders.filter(F.col("inconsistency_delivered_null") == 0)
orders = orders.drop("inconsistency_delivered_null")

### Order Itens

In [28]:
order_items.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)



In [29]:
order_items.show(5)

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18|12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51|199.9|        18.14|
+--------------------+-------------+------------

In [30]:
# Compound key verification
order_items = order_items.withColumn("compound_key", F.concat_ws("_", F.col("order_id"), F.col("order_item_id")))
verify_unique_key(order_items, "compound_key")

Column 'compound_key' is a unique key.


True

In [31]:
show_null_values(order_items)

+--------+-------------+----------+---------+-------------------+-----+-------------+------------+
|order_id|order_item_id|product_id|seller_id|shipping_limit_date|price|freight_value|compound_key|
+--------+-------------+----------+---------+-------------------+-----+-------------+------------+
|       0|            0|         0|        0|                  0|    0|            0|           0|
+--------+-------------+----------+---------+-------------------+-----+-------------+------------+



### Products

In [32]:
products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)



In [33]:
products.show(5)

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
|96bd76ec8810374ed...|        esporte_lazer|                 46|                       250|    

In [34]:
verify_unique_key(products, "product_id")

Column 'product_id' is a unique key.


True

In [35]:
show_null_values(products)

+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|         0|                  610|                610|                       610|               610|               2|                2|                2|               2|
+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+



In [36]:
# Treating null values

products = products.withColumn("product_category_name", F.when(F.col("product_category_name").isNotNull(), F.col("product_category_name")).otherwise("unknown"))
products = products.withColumn("product_name_lenght", F.when(F.col("product_name_lenght").isNotNull(), F.col("product_name_lenght")).otherwise(0))
products = products.withColumn("product_description_lenght", F.when(F.col("product_description_lenght").isNotNull(), F.col("product_description_lenght")).otherwise(0))
products = products.withColumn("product_photos_qty", F.when(F.col("product_photos_qty").isNotNull(), F.col("product_photos_qty")).otherwise(0))
products = products.withColumn("product_weight_g", F.when(F.col("product_weight_g").isNotNull(), F.col("product_weight_g")).otherwise(0))
products = products.withColumn("product_length_cm", F.when(F.col("product_length_cm").isNotNull(), F.col("product_length_cm")).otherwise(0))
products = products.withColumn("product_height_cm", F.when(F.col("product_height_cm").isNotNull(), F.col("product_height_cm")).otherwise(0))
products = products.withColumn("product_width_cm", F.when(F.col("product_width_cm").isNotNull(), F.col("product_width_cm")).otherwise(0))

show_null_values(products)

+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|         0|                    0|                  0|                         0|                 0|               0|                0|                0|               0|
+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+



### Order Payments

In [37]:
order_payments.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)



In [38]:
order_payments.show(5)

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
+--------------------+------------------+------------+--------------------+-------------+
only showing top 5 rows



In [39]:
show_null_values(order_payments)

+--------+------------------+------------+--------------------+-------------+
|order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------+------------------+------------+--------------------+-------------+
|       0|                 0|           0|                   0|            0|
+--------+------------------+------------+--------------------+-------------+



In [40]:
order_payments = order_payments.withColumn("compound_key", F.concat_ws("_", F.col("order_id"), F.col("payment_sequential")))
verify_unique_key(order_payments, "compound_key")

Column 'compound_key' is a unique key.


True

### Order Reviews

In [41]:
order_reviews.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: integer (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: timestamp (nullable = true)
 |-- review_answer_timestamp: timestamp (nullable = true)



In [42]:
order_reviews.show(5)

+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|                NULL|                  NULL| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|                NULL|                  NULL| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|                NULL|                  NULL| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|                NULL|  Recebi bem antes ...| 2017-04-21 00:00:00|   

In [43]:
show_null_values(order_reviews)

+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+
|review_id|order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+
|        0|       0|           0|               87656|                 58247|                   0|                      0|
+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+



### Sellers

In [44]:
sellers.printSchema()

root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: integer (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)



In [45]:
sellers.show(5)

+--------------------+----------------------+-----------------+------------+
|           seller_id|seller_zip_code_prefix|      seller_city|seller_state|
+--------------------+----------------------+-----------------+------------+
|3442f8959a84dea7e...|                 13023|         campinas|          SP|
|d1b65fc7debc3361e...|                 13844|       mogi guacu|          SP|
|ce3ad9de960102d06...|                 20031|   rio de janeiro|          RJ|
|c0f3eea2e14555b6f...|                  4195|        sao paulo|          SP|
|51a04a8a6bdcb23de...|                 12914|braganca paulista|          SP|
+--------------------+----------------------+-----------------+------------+
only showing top 5 rows



In [46]:
show_null_values(sellers)

+---------+----------------------+-----------+------------+
|seller_id|seller_zip_code_prefix|seller_city|seller_state|
+---------+----------------------+-----------+------------+
|        0|                     0|          0|           0|
+---------+----------------------+-----------+------------+



In [47]:
verify_unique_key(sellers, "seller_id")

Column 'seller_id' is a unique key.


True

### Product Category Name

In [48]:
product_category_name.printSchema()

root
 |-- product_category_name: string (nullable = true)
 |-- product_category_name_english: string (nullable = true)



In [49]:
product_category_name.show(5)

+---------------------+-----------------------------+
|product_category_name|product_category_name_english|
+---------------------+-----------------------------+
|         beleza_saude|                health_beauty|
| informatica_acess...|         computers_accesso...|
|           automotivo|                         auto|
|      cama_mesa_banho|               bed_bath_table|
|     moveis_decoracao|              furniture_decor|
+---------------------+-----------------------------+
only showing top 5 rows



In [50]:
show_null_values(product_category_name)

+---------------------+-----------------------------+
|product_category_name|product_category_name_english|
+---------------------+-----------------------------+
|                    0|                            0|
+---------------------+-----------------------------+



In [51]:
verify_unique_key(product_category_name, "product_category_name")

Column 'product_category_name' is a unique key.


True

## Data Ingestion

In [55]:
customers.toPandas().to_parquet("C:/GitHubJFF/olist_data/data/bronze/b_customers.parquet",engine="pyarrow",index=False)
geolocation.toPandas().to_parquet("C:/GitHubJFF/olist_data/data/bronze/b_geolocation.parquet",engine="pyarrow",index=False)
orders.toPandas().to_parquet("C:/GitHubJFF/olist_data/data/bronze/b_orders.parquet",engine="pyarrow",index=False)
order_items.toPandas().to_parquet("C:/GitHubJFF/olist_data/data/bronze/b_order_items.parquet",engine="pyarrow",index=False)
products.toPandas().to_parquet("C:/GitHubJFF/olist_data/data/bronze/b_products.parquet",engine="pyarrow",index=False)
order_payments.toPandas().to_parquet("C:/GitHubJFF/olist_data/data/bronze/b_order_payments.parquet",engine="pyarrow",index=False)
order_reviews.toPandas().to_parquet("C:/GitHubJFF/olist_data/data/bronze/b_order_reviews.parquet",engine="pyarrow",index=False)
sellers.toPandas().to_parquet("C:/GitHubJFF/olist_data/data/bronze/b_sellers.parquet",engine="pyarrow",index=False)
product_category_name.toPandas().to_parquet("C:/GitHubJFF/olist_data/data/bronze/b_product_category_name.parquet",engine="pyarrow",index=False)

In [56]:
spark.stop()